**In this assignment, you'll get to practice the concepts and skills covered in the course so far. The main objective of this assignment is to implement and use some of the tools, algorithms, and techniques to perform sentiment analysis on textual data..**



**Guidelines**
* Download `fake_news.csv` and `glove.6B.300d.txt` files from D2L. 
* Make sure to run all the code cells, otherwise you may get errors like `NameError` for undefined variables.
* Do not change variable names, delete cells or disturb other existing code. It may cause problems during evaluation.
* In some cases, you may need to add some code cells or new statements before or after the line of code containing the `???`.
* Use markdown cells to write your discussions and reflections. 

**Procedure**
* Save your work as `IPYNB` file named `Lab9.ipynb` and submit to D2L `Lab 9 - Word Embeddings - GloVe (Dropbox)` by the due date.
* As you go through this notebook, you will find the symbol `???` in certain places. To complete this assignment, you must replace all the `???` with appropriate values, expressions or statements to ensure that the notebook runs properly end-to-end.
* Include your response for `Part 1` and `Part 2` in this notebook. 

<div class="alert alert-block alert-info">

# Part 1: Activity 

</div>

1. Read the content of the `fake_news.csv` into a dataframe called `fake_news_df`. Then, perform the following on the dataframe:  
    1.1. Fill all nan values with an empty string (Whitespace).  
	1.2. Merge the fields title, author, and text into a single field called `combined_text` and append it to the dataframe. Note: remember to add whitespaces between the fields when combining them.  

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os

import string
import gzip
from collections import Counter
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pprint
from pprint import pprint

import watermark
%matplotlib inline
%load_ext watermark

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import re

import io


import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

import nltk 
from nltk.text import TextCollection
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer


import string
import gzip
from collections import Counter
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pprint
from pprint import pprint

import watermark
%matplotlib inline
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [3]:
fake_news_df = pd.read_csv("fake_news.csv")

In [4]:
fake_news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
fake_news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
print(fake_news_df['author'][50:60])# just a subset to get the pars that contains missing value.

50    Lance Schuttler
51                NaN
52        Nate Church
53        Dairy✓ᵀᴿᵁᴹᴾ
54                NaN
55    Lance Schuttler
56           Madeline
57         M.R. Islam
58            The Doc
59     Jeff Gordinier
Name: author, dtype: object


In [7]:
fake_news_df.fillna('',inplace=True)

In [8]:
fake_news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
fake_news_df['combined_text']=fake_news_df['title']+fake_news_df['author']+fake_news_df['text']

In [10]:
fake_news_df.head()

,id,title,author,text,label,combined_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredConsortiumnew...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [11]:
fake_news_df['combined_text'][1]#subset

'FLYNN: Hillary Clinton, Big Woman on Campus - BreitbartDaniel J. FlynnEver get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. 

2. Load the `glove.6B.300d.txt` GloVe vector into a dictionary called `embeddings_index`. In this dictionary, the keys are the `words` and the values are the `coefs` of these words from `glove.6B.300d.txt`  

In [12]:
import gensim

In [13]:
from gensim.scripts.glove2word2vec import glove2word2vec

embeddings_index={}
glove_filename = 'glove.6B.300d (2).txt'
glove_path = '/Users/aishatolatunji/Downloads/' + glove_filename
embeddings_index = glove_filename + '.word2vec'
glove2word2vec(glove_path, embeddings_index)

(400000, 300)

In [14]:
embeddings_index

'glove.6B.300d (2).txt.word2vec'

In [15]:
text = open('glove.6B.300d (2).txt','r')
words = text.read()

In [16]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(embeddings_index, binary=False)



3. Create a function called `sent2vec` that receives a sentence and creates a normalized vector for the whole sentences. This function must perform the following on the received sentence:  
	3.1. Converts the whole sentence into lowercase  
	3.2. Tokenizes the sentence into individual words or tokens and store them in a list called `tokens`  
    3.3. Removes stopwords from the `tokens`, `numbers`, `special characters`, and `puctuation` from the tokens and store the results in d list called `cleaned_tokens`  
	3.4. Gets the embeddings of all tokens in the `cleaned_tokens` and store them in a list called `word_embeddings`. Note: ignore any token that does not exist in the `embeddings_index` dictionary.  
	3.5. Converts the `word_embeddings` list into a numpy array called `emb_arr` and then calculates the normalized vector using the following code segment:  
```python
emb_arr = np.array(word_embeddings)
norm_vec = emb_arr.sum(axis=0)
if type(norm_vec) != np.ndarray:
        return np.zeros(300)
return norm_vec / np.sqrt((norm_vec ** 2).sum())
```

In [17]:
def sent2vec(sentence):
    sentence=sentence.lower()
    cleaned_sentence=cleanText(sentence)
    
    tokens = cleaned_sentence.split(' ')
    
    stop_words= set(stopwords.words('english'))
    cleaned_tokens=[word for word in tokens if word not in stop_words ]
    
    word_embeddings = [embeddings_index.get_vector(value) for value in cleaned_tokens
    if embeddings_index.has_index_for(value)]
    
    emb_arr=np.array(word_embeddings)
    norm_vec = emb_arr.sum(axis=0)
    if type(norm_vec)!= np.ndarray:
        return np.zeros(100)
    return norm_vec/np.sqrt((norm_vec ** 2).sum())

In [ ]:
???

def sent2vec(sentence):
    cleaned_tokens=[]
    word_embeddings=[]
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    punctuation = re.compile('<.*?>')
    cleantext = re.sub(punctuation, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    non_digits = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(non_digits)  
    cleaned_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    cleaned_tokens.append(cleaned_words)
    words = ' '.join([str(x) for x in cleaned_tokens])
    word_embeddings = gensim.models.Word2Vec(words, min_count = 1)
    emb_arr = np.array(word_embeddings)
    norm_vec = emb_arr.sum(axis=0)
    if type(norm_vec) != np.ndarray:
        return np.zeros(300)
    
    return norm_vec / np.sqrt((norm_vec ** 2).sum())
    

In [18]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aishatolatunji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
example=sample_df['combined_text'].map(sent2vec)

NameError: name 'sample_df' is not defined

In [23]:
example

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: combined_text, dtype: object

4. Create a numpy array called `fake_news_glove` that contain sentence vectors using the `sent2vec` function applied on the combined text field of the dataframe. This array will be used to generate the training and testing dataset for the following question. 

In [ ]:
???

In [18]:
fake_news_glove=fake_news_df['combined_text'].map(sent2vec)

NameError: name 'cleanText' is not defined

In [25]:
fake_news_glove=np.array(fake_news_glove)

In [30]:
print(fake_news_glove)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

5. Build any supervised learning model of your choice to classify news text into fake news or real news. Note: the supervised learning model could be a logistic regression, a decision tree, a random forest, etc.   

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer



In [31]:
fake_news_df.head()

,id,title,author,text,label,combined_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredConsortiumnew...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [35]:
labels = fake_news_df.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [36]:
x_train,x_test,y_train,y_test=train_test_split(fake_news_df['text'],labels,test_size=0.2,random_state=7)

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
tfdif_vectorizer = TfidfVectorizer(stop_words='english', max_df= 0.7)

tfdif_train= tfdif_vectorizer.fit_transform(x_train)
tfdif_test = tfdif_vectorizer.transform(x_test)

In [ ]:
# support vector machine 

A support vector machine is an unsupervised machine learning technique that analyze data and categorizes it into one ofthe two groups

In [43]:
from sklearn.svm import LinearSVC, SVC

In [50]:
from sklearn.svm import LinearSVC

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [52]:
svc=LinearSVC()
svc.fit(tfdif_train, y_train)

LinearSVC()

In [54]:
svc.predict(tfdif_test)

array([1, 1, 0, ..., 0, 0, 0])

In [79]:
y_pred_SVM=svc.predict(tfdif_test)
score = accuracy_score(y_test,y_pred_SVM)


6. Evaluate the model in terms of classification accuracy and discuss the results.   

In [81]:
print(f'Accuracy: {round(score*100,1)}%')

Accuracy: 96.7%


In [58]:
y_pred_SVM

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
???

In [ ]:
# Logistic Regression

Logistic regression is also another example of supervised machine learning technique, it's a go to algorith for choice of binary classification

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

In [63]:
model = LogisticRegression(solver='liblinear',random_state=0)
model.fit(tfdif_train,y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [82]:
y_pred_LR = model.predict(tfdif_test)
score = accuracy_score(y_test,y_pred_LR)

6. Evaluate the model in terms of classification accuracy and discuss the results.   

In [84]:
print(f'Accuracy: {round(score*100,1)}%')

Accuracy: 95.4%


In [66]:
y_pred_LR

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
# Random Forest

A random is also another example of supervised machine learning teechnique it's an ensemble classificationtechnique that is used in various application areas, it involves parallel ensembling, which involves fitting many decision tree classifiers in parallel on various data set sub-samples and using majority voting or averages to determine the conclusion or final result.

In [67]:
from sklearn.ensemble import RandomForestClassifier

In [68]:
clf = RandomForestClassifier()
clf.fit(tfdif_train, y_train)

RandomForestClassifier()

In [89]:
y_pred_RF=clf.predict(tfdif_test)
score = accuracy_score(y_test,y_pred_RF)

6. Evaluate the model in terms of classification accuracy and discuss the results.   

In [90]:
print(f'Accuracy: {round(score*100,1)}%')

Accuracy: 91.7%


In [91]:
y_pred_RF

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
# Decision tree

Decision tree is also an example of machine learning techique, it's used to split learning tasks and the tree is built by dividing the dataset into smaller groups until each division is clean.

In [71]:
from sklearn.tree import DecisionTreeClassifier
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)
classifier.fit(tfdif_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [87]:
y_pred_DT = classifier.predict(tfdif_test)
score = accuracy_score(y_test,y_pred_DT)


6. Evaluate the model in terms of classification accuracy and discuss the results.   

In [75]:
print(f'Accuracy: {round(score*100,1)}%')

Accuracy: 89.4%


In [88]:
y_pred_DT

array([1, 1, 0, ..., 0, 0, 0])

I also observed that support vector machine produced an accuracy of them all, while random forest tree also did a good job with an accuracy of 91.7% whithout overfitting the data.

<div class="alert alert-block alert-info">

# Part 2: Reflection
    
</div>

As a second step—after answering the questions, include the following:
1. A reflection of your experience performing the activity. 
2. A reflection on the importance of learning this activity.
**Note:** include your reflection in this notebook as markdown cells. 

???

  1. A reflection of your experience performing the activity. 


Working on this project was a bit streneous from the onset for me, i had struggles withobtaining the sent2vec which slowed my work as i thought i'd need it in modelling the fake news, i had to go back to my machine learning labs i had saved on my computer to get the work done, it was a beauting been able to reflect back to what i've learnt in machine learning, in the previous semester.And it also helped me recall ways in which i could further do better to my group project and in what ways i could go about it and further things i needed to include in the project.

2. A reflection on the importance of learning this activity.

After working on this project, i understood the use of combinning NLP and machine learning together properly to combat the significance of problem of fake news.

i used the term frequency vectorizer to convert the collection of text to a matrix TF-IDF features. which helped in obtaining the x_train and y_train. which is a statistical metric used to assess the importance words in the corpus.

i also used various supervised machine learning techiques to access which of the models better predict this data.which includes
a. svm:A support vector machine is an unsupervised machine learning technique that analyze data and categorizes it into one ofthe two groups with an accuracy of 96.7%
b. Logistic Regression:Logistic regression is also another example of supervised machine learning technique, it's a go to algorith for choice of binary classification with an accuracy of 95.4%
c. A random is also another example of supervised machine learning teechnique it's an ensemble classificationtechnique that is used in various application areas, it involves parallel ensembling, which involves fitting many decision tree classifiers in parallel on various data set sub-samples and using majority voting or averages to determine the conclusion or final result.with an accuracy of 91.7%
d. Decision tree is also an example of machine learning techique, it's used to split learning tasks and the tree is built by dividing the dataset into smaller groups until each division is clean. with an accuracy of 89.4%

I also observed that support vector machine produced an accuracy of them all, while random forest tree also did a good job with an accuracy of 91.7% whithout overfitting the data.

<div class="alert alert-block alert-info">

# Submission
    
</div>

Submit **Lab9.ipynb** to the **Lab 9 - Word Embeddings - GloVe (Dropbox)** on D2L by the due date. 

<div class="alert alert-block alert-info">

# Grading Rubric
    
</div>

|Criterion	|Excellent	|Good	|Average	|Below Average	|Poor	|No Attempt|
|:--	|:--	|:--	|:--	|:-- 	|:--	|:-- |
|**Part 1:** Activity-Question 1|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 2|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 3|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 4|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 5|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 6|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 2:** Reflection|**10 points**- Reflection clearly ties to the module content; experience and importance clearly laid out|**8 points**- Reflection mostly ties to the module content; experience & importance are discussed|**6 points**- Reflection ties minimally to the module content; experience & importance are discussed but not thoroughly|**4 points**- Reflection does not tie to the module content; experience & importance are minimally discussed|**2 points**- Minimal effort to tie to content; minimal effort to describe experience/ importance|**0 points**- Did not complete the reflection